In [3]:
import os
import glob

# Get all markdown files in the data/markdown directory
markdown_files = glob.glob("./data/markdown_results/*.md")

# Dictionary to store markdown contents
markdown_contents = {}

# Read each markdown file
for file_path in markdown_files:
    filename = os.path.basename(file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        markdown_contents[filename] = f.read()


In [10]:
from collections import Counter

# Find common prefixes
prefix_counter = Counter()
for content in markdown_contents.values():
    for i in range(len(content)):
        prefix = content[:i]
        if prefix:
            prefix_counter[prefix] += 1

# Find common suffixes            
suffix_counter = Counter()
for content in markdown_contents.values():
    for i in range(len(content)):
        suffix = content[-i:]
        if suffix:
            suffix_counter[suffix] += 1

max_prefix_count = max(prefix_counter.values())
max_prefix_count_max_length = max(len(prefix) for prefix in prefix_counter.keys() if prefix_counter[prefix] == max_prefix_count)
common_prefixes = [prefix for prefix in prefix_counter.keys() if prefix_counter[prefix] == max_prefix_count and len(prefix) == max_prefix_count_max_length]

max_suffix_count = max(suffix_counter.values())
max_suffix_count_max_length = max(len(suffix) for suffix in suffix_counter.keys() if suffix_counter[suffix] == max_suffix_count)
common_suffixes = [suffix for suffix in suffix_counter.keys() if suffix_counter[suffix] == max_suffix_count and len(suffix) == max_suffix_count_max_length]

# Create stripped version
stripped_markdown_contents = {}
for filename, content in markdown_contents.items():
    stripped = content
    for prefix in common_prefixes:
        if stripped.startswith(prefix):
            stripped = stripped[len(prefix):]
    for suffix in common_suffixes:
        if stripped.endswith(suffix):
            stripped = stripped[:-len(suffix)]
    stripped_markdown_contents[filename] = stripped

stripped_markdown_contents

{'docs_openpipe_ai_api_reference_post_updatemetadata.md': 'Ctrl K\n\nSearch...\n\nNavigation\n\nLogs\n\nUpdate Metadata\n\nPOST\n\n/\n\nlogs\n\n/\n\nupdate-metadata\n\nTry it\n\ncURL\n\nPython\n\nJavaScript\n\nPHP\n\nGo\n\nJava\n\nCopy\n\n```\ncurl --request POST \\\n  --url https://api.openpipe.ai/api/v1/logs/update-metadata \\\n  --header \'Authorization: Bearer <token>\' \\\n  --header \'Content-Type: application/json\' \\\n  --data \'{\n  "filters": [\\\n    {\\\n      "field": "<string>",\\\n      "equals": "<string>"\\\n    }\\\n  ],\n  "metadata": {}\n}\'\n```\n\n200\n\ndefault\n\nCopy\n\n```\n{\n  "matchedLogs": 123\n}\n```\n\n#### Authorizations\n\n[\u200b](https://docs.openpipe.ai/api-reference/post-updatemetadata#authorization-authorization)\n\nAuthorization\n\nstring\n\nheader\n\nrequired\n\nBearer authentication header of the form `Bearer <token>`, where `<token>` is your auth token.\n\n#### Body\n\napplication/json\n\n[\u200b](https://docs.openpipe.ai/api-reference/post-u

In [11]:
# Concatenate contents with filenames as headers
concatenated_text = ""
for filename, content in stripped_markdown_contents.items():
    concatenated_text += f"=== {filename} ===\n{content}\n\n"

print(concatenated_text)


=== docs_openpipe_ai_api_reference_post_updatemetadata.md ===
Ctrl K

Search...

Navigation

Logs

Update Metadata

POST

/

logs

/

update-metadata

Try it

cURL

Python

JavaScript

PHP

Go

Java

Copy

```
curl --request POST \
  --url https://api.openpipe.ai/api/v1/logs/update-metadata \
  --header 'Authorization: Bearer <token>' \
  --header 'Content-Type: application/json' \
  --data '{
  "filters": [\
    {\
      "field": "<string>",\
      "equals": "<string>"\
    }\
  ],
  "metadata": {}
}'
```

200

default

Copy

```
{
  "matchedLogs": 123
}
```

#### Authorizations

[​](https://docs.openpipe.ai/api-reference/post-updatemetadata#authorization-authorization)

Authorization

string

header

required

Bearer authentication header of the form `Bearer <token>`, where `<token>` is your auth token.

#### Body

application/json

[​](https://docs.openpipe.ai/api-reference/post-updatemetadata#body-filters)

filters

object\[\]

required

Showchild attributes

[​](https://docs.openp

In [1]:
print(
    """On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:\n\n\u2022 Miss Peach\n\u2022 Monsieur Brunette\n\u2022 Mr. Green\n\u2022 Professor Plum\n\u2022 Mrs. White\n\u2022 Colonel Mustard\n\u2022 Miss Scarlet\n\u2022 Mrs. Peacock\n\u2022 Sgt. Gray\n\u2022 Madame Rose\n\nAnd the following weapons were found on the premises:\n\n\u2022 Candlestick\n\u2022 Wrench\n\u2022 Lead Pipe\n\u2022 Revolver\n\u2022 Poison\n\u2022 Knife\n\u2022 Rope\n\u2022 Horseshoe\n\nThe murder could only have occured in one of the following rooms:\n\n01. Studio\n02. Gazebo\n03. Lounge\n04. Drawing Room\n05. Library\n06. Trophy Room\n07. Cloak Room\n08. Courtyard\n09. Kitchen\n10. Fountain\n11. Dining Room\n12. Carriage House\n13. Ballroom\n\nThe rooms are laid out as follows:\n\n  NN NN NN NN  \nW 01|02|03|04 E\nW 05|06|07|08 E\nW 09|10|11|12 E\nW 13|-|-|- E\n  SS SS SS SS  \n\nThe exact time of the murder is a bit uncertain, but it has been narrowed down to one of the following times:\n\n\u2022 08:00 PM\n\u2022 09:00 PM\n\u2022 10:00 PM\n\u2022 11:00 PM\n\u2022 12:00 AM\n\u2022 01:00 AM\n\u2022 02:00 AM\n\u2022 03:00 AM\n\nAt every time the suspects and Mr. Boddy either stayed in their current room or moved to an orthogonally adjacent room (north, south, east, or west). Weapons could be moved by suspects between rooms as well.\n\nEach suspect uniquely had one of the following possible motives for killing Mr. Boddy:\n\n\u2022 Jealousy\n\u2022 Ambition\n\u2022 Power\n\u2022 Betrayal\n\u2022 Anger\n\u2022 Fear\n\u2022 Greed\n\u2022 Hatred\n\u2022 Revenge\n\u2022 Pride\n\nFor the murder to occur, the murderer and Mr. Boddy must have been alone in a room with at least one weapon at some point in the night. Any clue about Mr. Boddy's whereabouts should be read as \"Mr. Boddy (dead or alive) ...\"\n\nThe available clues are as follows:\n\n- The murderer was in the Library at 12:00 AM\n- Mr. Green was at the Fountain at 12:00 AM\n- The Wrench was in the same room as Mr. Boddy at 10:00 PM\n- Mr. Boddy was in the Kitchen at 09:00 PM if and only if the suspect motivated by Fear was at the Fountain at 11:00 PM\n- Mr. Green was in the Trophy Room at 03:00 AM if and only if Mr. Boddy was in the Kitchen at 08:00 PM\n- Miss Scarlet was in the Library at 02:00 AM if and only if Mr. Green was in the Ballroom at 08:00 PM\n- Miss Scarlet was in the Ballroom at 08:00 PM if and only if Mr. Green was in the Trophy Room at 03:00 AM\n- Mrs. White moved from the Kitchen to the Library at 03:00 AM\n- Professor Plum and Madame Rose were in the Carriage House together at least once\n- The suspect motivated by Power moved the Poison from the Carriage House to the Courtyard at 11:00 PM\n- The suspect motivated by Hatred was in the Ballroom at 11:00 PM or the suspect motivated by Betrayal was in the Studio at 09:00 PM\n- Professor Plum was in the same room as Colonel Mustard at 08:00 PM\n- The suspect motivated by Pride was in the Trophy Room at 12:00 AM or the Horseshoe was in the Studio at 09:00 PM\n- Miss Scarlet was in the room just south of the Horseshoe at 12:00 AM\n- The Lead Pipe and Professor Plum were in the Courtyard together at least once\n- Mr. Boddy was murdered at 11:00 PM\n- The suspect motivated by Jealousy moved the Rope from the Trophy Room to the Library at 11:00 PM\n- Miss Scarlet was in the same room as Mrs. White at 08:00 PM\n- The murder weapon was in the same room as Miss Peach at 03:00 AM\n- The suspect motivated by Betrayal moved the Wrench from the Library to the Trophy Room at 03:00 AM\n- The suspect motivated by Ambition moved from the Lounge to the Gazebo at 11:00 PM\n- The suspect motivated by Anger moved from the Courtyard to the Cloak Room at 10:00 PM\n- The suspect motivated by Ambition moved from the Lounge to the Cloak Room at 03:00 AM\n- The suspect motivated by Greed was in the Kitchen at 01:00 AM\n- The Rope was in the room just north of the Knife at 08:00 PM\n- The Knife was in the same room as the Revolver at 09:00 PM\n- The Wrench was in the Carriage House at 11:00 PM or Mrs. White was in the Kitchen at 10:00 PM\n- The suspect motivated by Ambition moved from the Drawing Room to the Lounge at 09:00 PM\n- Miss Peach was in the Dining Room at 11:00 PM or the suspect motivated by Greed was in the Kitchen at 12:00 AM\n- The murderer was in the Library at 02:00 AM\n- Colonel Mustard moved from the Lounge to the Cloak Room at 11:00 PM\n- Madame Rose was in the Dining Room at 03:00 AM\n- Madame Rose was in the Dining Room at 02:00 AM\n- The suspect motivated by Anger moved from the Courtyard to the Carriage House at 12:00 AM\n- The Candlestick was in the Drawing Room at 10:00 PM\n- Mrs. Peacock moved from the Courtyard to the Carriage House at 12:00 AM\n- Mrs. Peacock moved the Poison from the Dining Room to the Carriage House at 10:00 PM\n- Mr. Green moved the Revolver from the Fountain to the Trophy Room at 02:00 AM\n- Mrs. Peacock was in the Courtyard at 01:00 AM\n- Colonel Mustard was in the room just south of Sgt. Gray at 01:00 AM\n- The suspect motivated by Betrayal was in the Ballroom at 09:00 PM if and only if the suspect motivated by Power was in the Carriage House at 10:00 PM\n- Miss Scarlet moved from the Ballroom to the Kitchen at 11:00 PM\n- Madame Rose was in the room just north of Mrs. Peacock at 08:00 PM\n- Miss Peach was in the Kitchen at 08:00 PM or the Horseshoe was in the Carriage House at 02:00 AM\n- Monsieur Brunette was in the Library at 01:00 AM\n- The suspect motivated by Hatred moved from the Kitchen to the Ballroom at 02:00 AM\n- Madame Rose moved from the Cloak Room to the Courtyard at 09:00 PM\n- Mr. Boddy was in the Kitchen at 09:00 PM\n- The suspect motivated by Pride moved from the Drawing Room to the Lounge at 10:00 PM\n- The Lead Pipe was in the Carriage House at 08:00 PM\n- The suspect motivated by Betrayal was in the Drawing Room at 02:00 AM or Miss Peach was in the Kitchen at 12:00 AM\n- The suspect motivated by Betrayal was in the Ballroom at 10:00 PM if and only if the suspect motivated by Power was in the Carriage House at 02:00 AM\n- The suspect motivated by Power was in the Carriage House at 03:00 AM\n- The murder weapon was in the Trophy Room at 08:00 PM\n- Mr. Green was in the Ballroom at 08:00 PM if and only if Professor Plum was in the Carriage House at 01:00 AM\n- Monsieur Brunette moved from the Cloak Room to the Trophy Room at 09:00 PM\n- Mrs. White and Monsieur Brunette were in the Library together at least once\n- Mr. Green moved from the Kitchen to the Fountain at 10:00 PM\n- Mrs. Peacock was in the room just north of Professor Plum at 03:00 AM\n- Madame Rose was in the Dining Room at 01:00 AM\n- Mr. Green was at the Fountain at 01:00 AM if and only if Professor Plum was in the Drawing Room at 08:00 PM\n- Mr. Green and the Knife were in the Trophy Room together at least once\n- Mr. Boddy was in the Dining Room at 10:00 PM or Mrs. Peacock was in the Courtyard at 02:00 AM\n- Miss Scarlet was in the same room as Mrs. White at 11:00 PM\n- The suspect motivated by Ambition was in the Gazebo at 12:00 AM\n- Miss Scarlet moved the Rope from the Library to the Trophy Room at 03:00 AM\n- The suspect motivated by Hatred moved from the Kitchen to the Ballroom at 10:00 PM\n- The suspect motivated by Power was in the same room as the Poison at 02:00 AM\n- Mrs. White moved from the Ballroom to the Kitchen at 09:00 PM\n- The Poison was in the same room as the Lead Pipe at 01:00 AM\n- The Wrench was in the same room as the Horseshoe at 12:00 AM\n- Miss Scarlet was in the Kitchen at 01:00 AM if and only if Professor Plum was in the Courtyard at 09:00 PM\n\nPlease answer the following question(s):\n\nA. Who murdered Mr. Boddy?\nB. What weapon did the murderer use?\nC. Where was the murder committed?\nD. Why did the murderer do it?\n\nAnd the following bonus question(s):\n\nE. Where was the suspect motivated by Revenge at 03:00 AM?\nF. Where was the Candlestick at 01:00 AM?\nG. Where was the suspect motivated by Ambition at 08:00 PM?\nH. Where was the Revolver at 08:00 PM?\n\nFill out your final answers in the following format:\n\nA. SUSPECT\nB. WEAPON\nC. ROOM\nD. MOTIVE\nE. ROOM\nF. ROOM\nG. ROOM\nH. ROOM\n\nBest of luck, detective."""
)

On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:

• Miss Peach
• Monsieur Brunette
• Mr. Green
• Professor Plum
• Mrs. White
• Colonel Mustard
• Miss Scarlet
• Mrs. Peacock
• Sgt. Gray
• Madame Rose

And the following weapons were found on the premises:

• Candlestick
• Wrench
• Lead Pipe
• Revolver
• Poison
• Knife
• Rope
• Horseshoe

The murder could only have occured in one of the following rooms:

01. Studio
02. Gazebo
03. Lounge
04. Drawing Room
05. Library
06. Trophy Room
07. Cloak Room
08. Courtyard
09. Kitchen
10. Fountain
11. Dining Room
12. Carriage House
13. Ballroom

The rooms are laid out as follows:

  NN NN NN NN  
W 01|02|03|04 E
W 05|06|07|08 E
W 09|10|11|12 E
W 13|-|-|- E
  S

In [1]:
from openai import AsyncOpenAI

client = AsyncOpenAI()
await client.chat.completions.create(
    model="o1-mini",
    messages=[{"role": "user", "content": "Develop an exhaustive rubric and rank all the presidents of the United States from best to worst."}],
)

ChatCompletion(id='chatcmpl-Avax6KV8klyDUhaylq4PeO1KiIE6s', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Ranking all the Presidents of the United States from best to worst is a complex and inherently subjective task. To approach this systematically, I have developed an exhaustive rubric that evaluates each president based on multiple key criteria. This rubric aims to provide a balanced and comprehensive assessment of each president's performance and legacy.\n\n**Ranking Rubric for U.S. Presidents**\n\nThe following criteria are used to evaluate each president, along with their respective weightings to reflect their importance:\n\n1. **Leadership and Vision (20 points)**\n   - Ability to provide clear direction and inspire the nation.\n   - Articulation of a compelling and effective vision for the country’s future.\n\n2. **Economic Management (15 points)**\n   - Success in promoting economic growth and stability.\n   - Effectivenes

In [1]:
704 / 64

11.0

In [6]:
import torch

class Attention(torch.nn.Module):
    def __init__(self, dim: int) -> None:
        super().__init__()
        self.wq = torch.nn.Linear(dim, dim, bias=False)
        self.wk = torch.nn.Linear(dim, dim, bias=False)
        self.wv = torch.nn.Linear(dim, dim, bias=False)

    def forward(self, x: torch.Tensor, mask: torch.Tensor | None = None) -> torch.Tensor:
        Q = self.wq(x)  # [batch_size, seq_len, dim]
        K = self.wk(x)  # [batch_size, seq_len, dim]
        V = self.wv(x)  # [batch_size, seq_len, dim]

        # Compute scaled dot-product attention
        attention = Q @ K.transpose(-2, -1)  # [batch_size, seq_len, seq_len]
        attention /= Q.shape[-1] ** 0.5  # Scale by sqrt of dimension
        
        if mask is not None:  # Check for mask presence
            attention += mask.to(attention.dtype) * -1e9
        
        attention = torch.softmax(attention, dim=-1)  # Apply softmax along seq_len

        # Weighted sum of values
        output = attention @ V  # [batch_size, seq_len, dim]
        return output


# Test the updated Attention module

# Create a random input tensor
batch_size = 2
seq_len = 5
dim = 8
x = torch.randn(batch_size, seq_len, dim)

# Instantiate the Attention module
attention_module = Attention(dim=dim)

# Test 1: Without Mask
output_no_mask = attention_module(x)

# Test 2: With Mask (causal mask example)
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).unsqueeze(0).repeat(batch_size, 1, 1) * -1e9
output_with_mask = attention_module(x, mask=mask)

# Output shapes
output_shapes = {
    "without_mask": output_no_mask.shape,
    "with_mask": output_with_mask.shape
}
output_shapes


{'without_mask': torch.Size([2, 5, 8]), 'with_mask': torch.Size([2, 5, 8])}